In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import requests # library to handle requests
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
!conda install -c conda-forge geocoder --yes
import geocoder # to get coordinates

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

## Loading Suburbs Dataframe 

In [2]:
df =pd.read_csv("suburbs.csv",index_col=0)
df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [3]:
df.shape

(40, 1)

## Using Geocoder to get the latitude and longitude of each location

In [4]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
coordinates = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [6]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [7]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']

In [8]:
# check the neighborhoods and the coordinates
df.head()

,Neighborhood,Latitude,Longitude
0,Andheri,19.118483,72.841774
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062930,72.926660
3,Bandra,19.054220,72.840190
4,Bhandup,19.145560,72.948560


In [9]:
df.shape

(40, 3)

In [10]:
# save the DataFrame as CSV file
df.to_csv("Neighborhood_Coordinates.csv", index=False)

## Getting Mumbai's Coordinates to create a map

In [11]:
# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [12]:

map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
map

In [13]:
# save the map as HTML file
map.save('map.html')

## Denifing Foursquare Credentials

In [1]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


## Getting Top 100 venues

In [15]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [16]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2668, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118483,72.841774,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118483,72.841774,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118483,72.841774,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118483,72.841774,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118483,72.841774,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop


## Looking at the venues count for each location

In [17]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,16,16,16,16,16,16
Baiganwadi,10,10,10,10,10,10
Bandra,100,100,100,100,100,100
Bhandup,24,24,24,24,24,24
Borivali,97,97,97,97,97,97
Charkop,55,55,55,55,55,55
Chembur,96,96,96,96,96,96
Dahisar,65,65,65,65,65,65


In [18]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 195 uniques categories.


In [19]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:100]

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop',
       'Falafel Restaurant', 'Coffee Shop', 'Sandwich Place', 'Pub',
       'Fast Food Restaurant', 'Pizza Place', 'Juice Bar', 'Multiplex',
       'Snack Place', 'Café', 'Breakfast Spot', 'Seafood Restaurant',
       'Cocktail Bar', 'Chinese Restaurant', 'American Restaurant', 'Bar',
       'Gym / Fitness Center', 'Electronics Store', 'Department Store',
       'Diner', 'BBQ Joint', 'Asian Restaurant', 'Lounge', 'Spa',
       "Women's Store", 'Liquor Store',
       'Residential Building (Apartment / Condo)',
       'Vegetarian / Vegan Restaurant', 'Food Truck', 'Smoke Shop',
       'Park', 'Fish Market', 'Martial Arts Dojo', 'Athletics & Sports',
       'Hotel', 'Tea Room', 'Italian Restaurant', 'Food', 'Plaza',
       'Sports Bar', 'Platform', 'Paper / Office Supplies Store',
       'Food & Drink Shop', 'Music Venue', 'Hot Dog Joint',
       'Fried Chicken Joint', 'Gym', 'Dessert Shop', 'Sports Club',
       'Gourmet Shop', 'Kor

## For Coffee Shop

### Since we are interested in finding out more information on the coffee shops we will now focus on it

In [20]:
coffee_df= venues_df[venues_df['VenueCategory']=='Coffee Shop']

In [21]:
coffee_df.shape

(108, 7)

In [22]:
coffee_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
4,Andheri,19.118483,72.841774,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop
99,Andheri,19.118483,72.841774,Cafe Coffee Day,19.130092,72.837659,Coffee Shop
106,Anushakti Nagar,19.042830,72.927340,Cafe X0,19.047971,72.931308,Coffee Shop
124,Baiganwadi,19.062930,72.926660,Cafe X0,19.047971,72.931308,Coffee Shop
197,Bandra,19.054220,72.840190,"Di Bella Coffee - Title Waves, Bandra",19.061029,72.832521,Coffee Shop


## Looking at the number of coffee shops in each suburb

In [23]:
group_df=coffee_df.groupby(['Neighborhood']).count()

In [24]:
group_df.head(20)

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,2,2,2,2,2,2
Anushakti Nagar,1,1,1,1,1,1
Baiganwadi,1,1,1,1,1,1
Bandra,2,2,2,2,2,2
Borivali,2,2,2,2,2,2
Charkop,3,3,3,3,3,3
Chembur,2,2,2,2,2,2
Dahisar,2,2,2,2,2,2
Devipada,2,2,2,2,2,2


## Encoding the column values

In [25]:
# one hot encoding
df_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

print(df_onehot.shape)
df_onehot.head()

(2668, 196)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0

In [26]:
df_grouped = df_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(df_grouped.shape)
df_grouped

(40, 196)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.010000,0.00,0.020000,0.000000,0.040000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.00,0.050000,0.000000,0.01,0.020000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.0000,0.00000,0.00,0.000000,0.00000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.01,0.00,0.000000,0.010000,0.00,0.01000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.040000,0.170000,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.030000,0.00,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000,0.01,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.050000,0.0

In [27]:
len(df_grouped[df_grouped["Coffee Shop"] > 0])

35

## Creating a dataframe that only contains Coffe Shop Data

In [28]:
df_coffee = df_grouped[["Neighborhoods","Coffee Shop"]]

In [29]:
df_coffee.head()

,Neighborhoods,Coffee Shop
0,Andheri,0.0200
1,Anushakti Nagar,0.0625
2,Baiganwadi,0.1000
3,Bandra,0.0200
4,Bhandup,0.0000


## Using K-Means Clustering for dividing the data into two Clusters

In [30]:
# set number of clusters
kclusters = 2

df_clustering = df_coffee.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [31]:

df_merged = df_coffee.copy()

# add clustering labels
df_merged["Cluster Labels"] = kmeans.labels_

In [32]:
df_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
df_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Andheri,0.0200,0
1,Anushakti Nagar,0.0625,1
2,Baiganwadi,0.1000,1
3,Bandra,0.0200,0
4,Bhandup,0.0000,0


In [33]:
df_merged = df_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

In [34]:
print(df_merged.shape)
df_merged.head() 

(40, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Andheri,0.0200,0,19.118483,72.841774
1,Anushakti Nagar,0.0625,1,19.042830,72.927340
2,Baiganwadi,0.1000,1,19.062930,72.926660
3,Bandra,0.0200,0,19.054220,72.840190
4,Bhandup,0.0000,0,19.145560,72.948560


In [35]:
# sort the results by Cluster Labels
print(df_merged.shape)
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged

(40, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Andheri,0.020000,0,19.118483,72.841774
36,Vikhroli,0.040000,0,19.111090,72.927810
31,Shil Phata,0.000000,0,19.146580,73.040050
30,Seven Bungalows,0.010000,0,19.128560,72.820850
29,Pestom sagar,0.037037,0,19.070630,72.902200
28,Mumbra,0.000000,0,19.190560,73.022670
27,Mulund,0.013333,0,19.171850,72.955640
26,Mogra Village,0.000000,0,26.135300,73.054840
25,Mira Road,0.036145,0,19.074161,72.861670
24,"Matharpacady, Mumbai",0.019231,0,19.044920,72.867205


## Plotting the results of the clustering 

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Saving the map created

In [37]:
map_clusters.save('map_clusters.html')

## Looking at individual clusters

In [38]:
df_merged.loc[df_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Andheri,0.020000,0,19.118483,72.841774
36,Vikhroli,0.040000,0,19.111090,72.927810
31,Shil Phata,0.000000,0,19.146580,73.040050
30,Seven Bungalows,0.010000,0,19.128560,72.820850
29,Pestom sagar,0.037037,0,19.070630,72.902200
28,Mumbra,0.000000,0,19.190560,73.022670
27,Mulund,0.013333,0,19.171850,72.955640
26,Mogra Village,0.000000,0,26.135300,73.054840
25,Mira Road,0.036145,0,19.074161,72.861670
24,"Matharpacady, Mumbai",0.019231,0,19.044920,72.867205


In [39]:
df_merged.loc[df_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
12,Ghatkopar,0.073171,1,19.086477,72.908956
37,Wadala,0.052083,1,19.017160,72.858130
1,Anushakti Nagar,0.062500,1,19.042830,72.927340
35,Vashi,0.080000,1,19.084700,72.904840
34,Tilak Nagar (Mumbai),0.058824,1,18.996160,72.852810
33,Thakur village,0.048387,1,19.210200,72.875410
32,"Sion, Mumbai",0.044118,1,19.046460,72.863130
2,Baiganwadi,0.100000,1,19.062930,72.926660
6,Charkop,0.054545,1,19.208660,72.826120
23,Mankhurd,0.076923,1,19.048530,72.932200


## For Restuarant

### The above process will be repeated to find clusters from the restaurant data

In [40]:
res_df= venues_df[venues_df['VenueCategory']=='Restaurant']

In [41]:
res_df.shape

(73, 7)

In [42]:
res_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
140,Bandra,19.05422,72.84019,Candies,19.049709,72.829785,Restaurant
215,Bandra,19.05422,72.84019,Gondola's,19.062725,72.828177,Restaurant
231,Bhandup,19.14556,72.94856,Gopala's Next,19.156164,72.937616,Restaurant
232,Bhandup,19.14556,72.94856,The Good Vibes Café,19.160218,72.946772,Restaurant
262,Borivali,19.22936,72.85751,Club Aquaria,19.240253,72.848016,Restaurant


In [43]:
group_df=res_df.groupby(['Neighborhood']).count()

In [44]:
group_df.head(20)

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Bandra,2,2,2,2,2,2
Bhandup,2,2,2,2,2,2
Borivali,6,6,6,6,6,6
Charkop,1,1,1,1,1,1
Chembur,3,3,3,3,3,3
Dahisar,3,3,3,3,3,3
Devipada,3,3,3,3,3,3
Eastern Suburbs (Mumbai),2,2,2,2,2,2
Ghatkopar,3,3,3,3,3,3


In [45]:
len(df_grouped[df_grouped["Restaurant"] > 0])

28

In [46]:
df_res = df_grouped[["Neighborhoods","Restaurant"]]

In [47]:
df_res.head()

,Neighborhoods,Restaurant
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.020000
4,Bhandup,0.083333


In [48]:
# set number of clusters
kclusters = 2

df_clustering = df_res.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0], dtype=int32)

In [49]:

df_merged_res = df_res.copy()

# add clustering labels
df_merged_res["Cluster Labels"] = kmeans.labels_

In [50]:
df_merged_res.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
df_merged_res.head()

,Neighborhood,Restaurant,Cluster Labels
0,Andheri,0.000000,0
1,Anushakti Nagar,0.000000,0
2,Baiganwadi,0.000000,0
3,Bandra,0.020000,0
4,Bhandup,0.083333,1


In [51]:
df_merged_res = df_merged_res.join(df.set_index("Neighborhood"), on="Neighborhood")

In [52]:
print(df_merged_res.shape)
df_merged_res.head() 

(40, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
1,Anushakti Nagar,0.000000,0,19.042830,72.927340
2,Baiganwadi,0.000000,0,19.062930,72.926660
3,Bandra,0.020000,0,19.054220,72.840190
4,Bhandup,0.083333,1,19.145560,72.948560


In [53]:
# sort the results by Cluster Labels
print(df_merged_res.shape)
df_merged_res.sort_values(["Cluster Labels"], inplace=True)
df_merged_res

(40, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
37,Wadala,0.010417,0,19.017160,72.858130
36,Vikhroli,0.020000,0,19.111090,72.927810
33,Thakur village,0.032258,0,19.210200,72.875410
32,"Sion, Mumbai",0.000000,0,19.046460,72.863130
31,Shil Phata,0.000000,0,19.146580,73.040050
30,Seven Bungalows,0.010000,0,19.128560,72.820850
27,Mulund,0.013333,0,19.171850,72.955640
26,Mogra Village,0.000000,0,26.135300,73.054840
25,Mira Road,0.012048,0,19.074161,72.861670


In [54]:
# create map
map_clusters_res = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged_res['Latitude'], df_merged_res['Longitude'], df_merged_res['Neighborhood'], df_merged_res['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_res)
       
map_clusters_res

In [55]:
map_clusters_res.save('map_clusters_res.html')

In [56]:
df_merged_res.loc[df_merged_res['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
37,Wadala,0.010417,0,19.017160,72.858130
36,Vikhroli,0.020000,0,19.111090,72.927810
33,Thakur village,0.032258,0,19.210200,72.875410
32,"Sion, Mumbai",0.000000,0,19.046460,72.863130
31,Shil Phata,0.000000,0,19.146580,73.040050
30,Seven Bungalows,0.010000,0,19.128560,72.820850
27,Mulund,0.013333,0,19.171850,72.955640
26,Mogra Village,0.000000,0,26.135300,73.054840
25,Mira Road,0.012048,0,19.074161,72.861670


In [57]:
df_merged_res.loc[df_merged_res['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
22,Mahavir Nagar (Kandivali),0.050000,1,19.211952,72.837542
8,Dahisar,0.046154,1,19.250030,72.859080
12,Ghatkopar,0.036585,1,19.086477,72.908956
28,Mumbra,0.166667,1,19.190560,73.022670
29,Pestom sagar,0.049383,1,19.070630,72.902200
5,Borivali,0.061856,1,19.229360,72.857510
4,Bhandup,0.083333,1,19.145560,72.948560
38,Western Suburbs (Mumbai),0.035714,1,19.197000,72.827630
34,Tilak Nagar (Mumbai),0.058824,1,18.996160,72.852810
35,Vashi,0.050000,1,19.084700,72.904840


## Using Cluster 0 of both the coffee shop data and restaurant data to find common suburbs

In [58]:
coffee_cluster = df_merged.loc[df_merged['Cluster Labels'] == 0]

In [59]:
res_cluster=df_merged_res.loc[df_merged_res['Cluster Labels'] == 0]

### Using intersection to find common neighborhoods

In [63]:
a=set(coffee_cluster['Neighborhood']).intersection(set(res_cluster['Neighborhood']))
a

{'Andheri',
 'Bandra',
 'Chembur',
 'Devipada',
 'Dombivli',
 'Jogeshwari',
 'Kanjurmarg',
 'Kausa',
 'Matharpacady, Mumbai',
 'Mira Road',
 'Mogra Village',
 'Mulund',
 'Seven Bungalows',
 'Shil Phata',
 'Vikhroli'}

In [66]:
common_df = pd.DataFrame(a,columns={"Suburb"})

In [67]:
common_df

,Suburb
0,Mulund
1,Kanjurmarg
2,"Matharpacady, Mumbai"
3,Andheri
4,Seven Bungalows
5,Kausa
6,Chembur
7,Mira Road
8,Jogeshwari
9,Devipada
